# Before running the following client side program: 

- first run "eva_server" in terminal with you python venv(i.e. test_eva_db you just created). 

- run this ipynb under the same python venv as that of server

In [ ]:
!wget -nc "https://raw.githubusercontent.com/sashiko-345/eva/master/dog.jpg"
!wget -nc https://www.dropbox.com/s/k00wge9exwkfxz6/ua_detrac.mp4
!wget "https://raw.githubusercontent.com/sashiko-345/eva/master/eva/udfs/camera_trap_animal_detector.py" 
!mv 'camera_trap_animal_detector.py' 'downloaded_camera_trap_animal_detector.py'

!pip install timm


In [1]:
# first run "eva_server" in terminal with you python venv(i.e. test_eva_db you just created)

# run this ipynb under the same python venv as that of server

# connect to server
from eva.server.db_api import connect
import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '127.0.0.1', port = 5435)
cursor = connection.cursor()

In [8]:
# load
cursor.execute('LOAD FILE "dog.jpg" INTO singlePic;')
response = cursor.fetch_all()
print(response)

@status: -1
@batch: None
@error: Failed to find a video file at location: dog.jpg


In [11]:
# load
cursor.execute('LOAD FILE "/Users/dawndawn/CS6422/trap-cls/eva/ua_detrac.mp4" INTO singleVid;')
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  Video successfully added at location: /Users/d...
@batch_size: 1
@identifier_column: None
@query_time: 0.02857986599974538


In [5]:
drop_udf_query = """DROP UDF AnimalDetectorPlus;
        """
cursor.execute(drop_udf_query)
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                              0
0  UDF AnimalDetectorPlus successfully dropped
@batch_size: 1
@identifier_column: None
@query_time: 0.06859839700000236


In [6]:
import os
# udf_path = os.getcwd()
udf_path = '/Users/dawndawn/CS6422/trap-cls/eva/eva/udfs/camera_trap_animal_detector.py'
create_udf_query = f"""CREATE UDF IF NOT EXISTS AnimalDetectorPlus
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (labels NDARRAY STR(ANYDIM),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  Classification
                  IMPL  '{udf_path}';
        """
# load
cursor.execute(create_udf_query)
response = cursor.fetch_all()
print(response)

@status: 0
@batch: Batch Object:
@dataframe:                                                    0
0  UDF AnimalDetectorPlus successfully added to t...
@batch_size: 1
@identifier_column: None
@query_time: 1.0238103379999899


In [7]:
# query dog with animal detector
cursor.execute('''SELECT id, AnimalDetectorPlus(data) FROM singleVid WHERE id < 1''')
response = cursor.fetch_all()
print(response)
# where id does not work for uploaded picture, only work for videos

@status: 0
@batch: Batch Object:
@dataframe:    singlevid.id animaldetectorplus.labels animaldetectorplus.scores
0             0                   [blank]            [0.7507480383]
@batch_size: 1
@identifier_column: None
@query_time: 9.597760284000003


In [32]:
from eva.udfs.camera_trap_animal_detector import AnimalDetectorPlus
from PIL import Image
img = Image.open('/Users/dawndawn/Desktop/cat.jpg')

In [33]:
detector = AnimalDetectorPlus()

In [35]:
from torchvision import transforms

composed = transforms.Compose([transforms.ToTensor()])
tensor = composed(img)

In [38]:
detector.forward(tensor)

/Users/dawndawn/CS6422/trap-cls/eva/eva/udfs/camera_trap_animal_detector.py:343: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outcome = outcome.append({'labels': self.labels, 'scores': frame_output}, ignore_index=True,)


,labels,scores
0,"[monkey_prosimian, rodent, human, mongoose, le...","[0.99999976, 0.19434793, 2.5042219e-08, 2.2078..."
